### Introduction: Applying Computational Thinking to Personal Finance and Portfolio Analysis
In an era where financial independence is closely tied to the ability to make strategic investment decisions, understanding the mechanics of saving and investing has become a critical life skill. Yet, while many individuals understand the importance of saving, few have the tools or knowledge required to evaluate how different financial strategies—and especially investment decisions—impact long-term outcomes. This tutorial addresses that gap by introducing students to essential concepts in personal finance and portfolio analysis through a computational and data-driven approach.

This tutorial begins by building a foundation in core savings principles. Learners will explore the functionality of common financial instruments such as high-yield savings accounts, 401(k)s, and Roth IRAs. Through interactive inputs and simulations, students will see how interest rates, compound growth, and tax advantages affect returns over time. These early exercises aim to demonstrate the value of starting early, contributing consistently, and choosing the right savings vehicles for one’s personal financial goals.

Building on this foundation, the tutorial then transitions into a more advanced, investment-focused module: constructing and evaluating a personalized stock portfolio. Students will be guided through the process of:
- Selecting their own breakdown of stocks to form a diversified portfolio based on individual risk preferences or interests.

- Evaluating the strength of their chosen portfolio using historical data and key financial metrics.

- Simulating potential future outcomes through Monte Carlo simulations, which model thousands of possible future scenarios to account for uncertainty and market volatility.

- Optimizing the stock allocation using algorithmic methods that balance return potential with risk, aiming for the most efficient use of investment capital.

- Rechecking the portfolio’s strength after optimization, to reflect the improvements made and demonstrate the impact of computational adjustments on financial performance.


Throughout this tutorial, students will employ principles from computer science—such as data modeling, simulations, algorithmic thinking, and data visualization—to analyze financial data and make informed decisions. By the end of the tutorial, students will have developed not only a deeper understanding of financial literacy but also a practical appreciation for how computational tools can empower individuals to manage their personal wealth more effectively.

This interdisciplinary exploration is designed to equip learners with a strong foundation in both personal finance and applied computation—skills that are increasingly essential in today’s data-rich, financially complex world.

### Disclaimer

**No Guarantee of Accuracy:** While Isabel, Luna, and Nirantheri strive to provide accurate and up-to-date information, they do not guarantee the accuracy, completeness, or reliability of any content. Users should independently verify any information before making financial or investment decisions based on it.


**Investment Risks:** Investing involves inherent risks, including but not limited to market fluctuations, economic uncertainty, geopolitical events, and individual asset performance. Past performance is not indicative of future results, and no content provided implies a guarantee of investment success.

**Limitation of Liability:** Mentions of specific financial products, services, companies, or securities within the content do not constitute endorsements or recommendations. Users are responsible for conducting their own research and due diligence.

**No Liability:** Isabel, Luna, Nirantheri, and their affiliates, partners, or contributors shall not be held liable for any loss, damage, or expense resulting from the use of or reliance on the provided information. All investment decisions made based on this content are the sole responsibility of the user.

**Disclaimer Updates:** This disclaimer is subject to change without notice. Users are responsible for reviewing it periodically to stay informed of any updates.



### Required Packages
If the "Module not found" error shows up, use the requirements.txt file to download the missing packages.

```
pip install -r /path/to/requirements.txt
```

In [1]:
from IPython.display import display

import matplotlib.pyplot as plt
%matplotlib inline

#alphabetical
import altair as alt
from curl_cffi import requests
import ipywidgets as widgets
import numpy as np
import pandas as pd
import quantstats.stats as stats
from scipy.optimize import minimize
from scipy.stats import norm
import yfinance as yf



### Selecting Stocks

First, you can select the stocks in your simulated portfolio. Ctrl+ Click the stocks that you want in your portfolio. Remember that a more diverse portfolio is better, so select at least 5 stocks to best model your findings.


If you'd like to create a custom allocation of stocks, skip to the section [Custom Allocations](#custom-allocation)

We used ipywidget documentation for [Checkboxes](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html#Checkbox) and [SelectMultiple](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html#SelectMultiple)


In [2]:
codes = {"Apple": 'AAPL',
        "Costco": 'COST',
        "Microsoft": 'MSFT',
        "Google": 'GOOG',
        "Starbucks":'SBUX',
        "Tyson Foods": 'TSN',
        "Intel Corp":'INTC',
        "Walmart": 'WMT',
        "Amazon":'AMZN',
        "BJ's Wholesale Club":'BJ',
        "Toyota":'TM',
        "S&P 500": '^GSPC',
        "Dow Jones Industrial": '^DJI',
        "Nasdaq": '^IXIC',
        "Vanguard S&P 500 ETF": 'VOO',
        "Fidelity 500 Index Fund": 'FXAIX',
        "Gold":'GLD'}

codes.keys()


choices = widgets.SelectMultiple(
    options=codes.keys(),
    value=[],
    rows=20,
    description='Stocks',
    disabled=False
)

display(choices)

SelectMultiple(description='Stocks', options=('Apple', 'Costco', 'Microsoft', 'Google', 'Starbucks', 'Tyson Fo…

### Picking The Breakdown of Stocks

<!-- clarifying what the breakdown of stocks is, why are we doing this -->

Without rerunning the previous code chunk, run the next one and input your allocation amounts as a decimal ex (0.3 or .5). Make sure they add up to 1! This step will allow you to invest in each stock proportionally.
<!--
[setup using the above code chunk](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html#Tabs) -->

This was setup using documentation for [Tabs](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html#Tabs)

In [3]:
# get codes for further down
portfolio = []

for i in range(len(choices.value)):
    portfolio.append(codes[choices.value[i]])

# pick distribution

tab_contents = ["Value"] * len(choices.value)
children = [widgets.Text(description=name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
tab.titles = choices.value

tab

Tab()

The following code is to match the allocation amounts to the stock codes given the inputs from above. You don't need to do anything here!

In [4]:
# matching allocation amounts to stock codes given inputs from above

allocations = {}

for i in range(len(tab.children)):
    allocations[portfolio[i]]=tab.children[i].value


### Custom Allocation 

Feel free to create your own allocation dictionary. Make sure each entry is of the format "STOCK TICKER": allocation amount. Fill it in as you desire and click run to make sure that your custom allocations are used, even if you didn't run the above lines. Otherwise our default set of stocks will be used! Do not run this cell unless you're planning on using a custom amount-- otherwise the default values we used will be input instead.

NOTE: for index funds, you have to prefix them with "^"

In [5]:
# Here's an example of what your dictionary can look like

# {'AAPL':'0.4', 'MSFT':'0.3',  '^IXIC':'0.3'}

allocations = {}

### Pulling Stock Data History Using the YahooFinance API

Using the list of stocks in our dictionary, the below code will grab the data for each stock.

NOTE: the session line is a temporary fix for a new rate-limiting error that yahoofinance api are running into recently, from [Github](https://github.com/ranaroussi/yfinance/issues/2422#issuecomment-2840774505) 

In [6]:
if len(portfolio)==0 or len(allocations)==0: # for default values
    allocations={'AAPL':'0.1', 'MSFT':'0.1',  '^IXIC':'0.1', 'COST': '0.1', 'WMT': '0.1', 'AMZN':'0.1', '^GSPC':'0.1', 'SBUX':'0.1', 'TSN':'0.1','INTC':'0.1'}
    portfolio = list(allocations.keys())

# This is so that the rate limiting error will not occur 
session = requests.Session(impersonate="chrome",timeout=5)

testList=list(allocations.keys())
testString = ' '.join(testList)

tickersPull = yf.Tickers(testString, session=session)

#in the dictionary dataframes, each ticker is indexed by its ticker as seen in the testList (auto_adjust accounts for splits and dividends)
dataframes = {}
for x in testList:
    ticker = yf.Ticker(x, session=session)
    dataframes[x] = ticker.history(period='1mo', start='2015-01-01', auto_adjust=True)

#### How do we calculate returns?

<!-- have we defined returns at this point? -->

To calculate the returns, we want to find the adjusted value of each stock amount from the date of the initial investment. We can write a function to do this with a set of inputs-- an initial portfolio value, the set of dataframes for each stock, and the breakdown of allocations  <!-- and perhaps even an initial date of investment.  -->

We used [this resource](https://blog.mlq.ai/python-for-finance-portfolio-optimization/) to help with this section.

In [7]:
def calculate_returns(initial_portfolio_val, dataframe, allocations):
    all_pos_vals = []

    for stock_name in dataframe:
        # grab the dataframe for a single stock
        stock_df = dataframe[stock_name]
        # create normed return column
        stock_df['Normed Return'] = stock_df['Close'] /stock_df.iloc[0]['Close']

        # use normed return to adjust the percentage of portfolio held
        allocation = float(allocations[stock_name])  # Convert allocation to float
        stock_df['Allocation'] = stock_df['Normed Return']*allocation

        # find value of stock at each date
        stock_df['Position Value'] = stock_df['Allocation']*initial_portfolio_val

        # add to list of all position values
        all_pos_vals.append(stock_df['Position Value'])

    # concatenate the list of position values
    portfolio_val = pd.concat(all_pos_vals, axis=1)

    # set the column names
    portfolio_val.columns = portfolio

    # add a total portfolio column
    portfolio_val['Total'] = portfolio_val.sum(axis=1)

    # changing date to column not index
    portfolio_val = portfolio_val.reset_index()

    return portfolio_val



Then we can write a function which prints the initial and final allocation amounts and creates a graphical representation of each of the stocks' growths over time as well as the portfolio's overall growth. To get an overall number rather than using every single data point, we can use a resampling function to grab the last day of each month in the data. 

Then we can create a graphing function which can graph both the total value of the portfolio over time as well as the breakdown of each stock's value while showing the value as you hover over it.

In [8]:
def graph_returns(returns, show_total, show_individual):

    # resampling values
    portfolio_val = returns.resample('ME', on='Date').first().reset_index()

    initial_amts = returns.iloc[0][1:]
    ending_amts = returns.iloc[-1][1:]
    initial_amts = initial_amts.reset_index()
    ending_amts = ending_amts.reset_index()

    amts = pd.merge(initial_amts, ending_amts)
    amts.columns = ["Date", "Initial Amounts", "Ending Amounts"]

    amts["Change in Value"] = amts["Ending Amounts"]-amts["Initial Amounts"]
    amts[["Initial Amounts", "Ending Amounts", "Change in Value"]] = amts[["Initial Amounts", "Ending Amounts", "Change in Value"]].apply(pd.to_numeric).round(2)
    print(amts)

    if show_total:
        # only need the total column
        data = portfolio_val.drop(portfolio, axis=1)
        pivoted_data_total = data.melt(id_vars="Date", var_name="Stock", value_name="Value")

        grapher(pivoted_data_total).display()



    if show_individual:
        pivoted_data_individual = portfolio_val.melt(id_vars="Date", value_vars = portfolio, var_name="Stock", value_name="Value")

        grapher(pivoted_data_individual).display()



def grapher(pivoted_data):
        # for stock vs total
        portfolio = pivoted_data['Stock'].unique()

        # set hovering conditions

        nearest = alt.selection_point(nearest=True, on="mouseover",fields=["Date"], empty=False)

        when_near = alt.when(nearest)

        # create basis of the graph
        basic_graph = alt.Chart(pivoted_data).mark_line().encode(
            alt.X("Date:T", title="Date"),
            alt.Y("Value:Q", title="Portfolio Value"),
            alt.Color("Stock:N", title="Stock"),
        ).interactive()

        # Draw points on the line, and if I am "near" then make them visible
        points = basic_graph.mark_point().encode(
            opacity=when_near.then(alt.value(1)).otherwise(alt.value(0))
        )

        # add rule so that a line shows up where the mouse is, and shows every data point at the mouseover.
        rules = alt.Chart(pivoted_data).transform_pivot(
            "Stock",
            value="Value",
            groupby=["Date"]
        ).mark_rule(color="gray").encode(
            x="Date:T",
            opacity=when_near.then(alt.value(0.3)).otherwise(alt.value(0)),
            tooltip=[
            alt.Tooltip("Date:T", title="Date"),
            *[alt.Tooltip(stock, type="quantitative", format=".2f") for stock in portfolio],
            ],
        ).add_params(nearest)


        # Put the data together
        chart = alt.layer(
            basic_graph, points, rules
        ).properties(
            width=600, height=300, # because the auto-size is quite small
            title="Portfolio Value Over Time"  
        )
        return chart


Now that we've created a projection of how our data has been growing over time, let's see what to expect!

### Visualizing Your Portfolio

The initial and ending values of the portfolio are printed for each stock and for the total amount.

The plots can show how your portfolio value evolved over time using your initial stock allocations. Feel free to alter the initial investment amount if you want to get an idea of your own personal finances.

- The **"Total" line** represents the entire portfolio’s value.
- The **colored lines** show how much each individual stock contributed to the total.
- **Hover over** the chart to see daily values and compare how stocks performed relative to each other.

Questions to Ask Yourself:
- Consistency: Do values rise steadily or fluctuate a lot?
- Dominance: Are certain stocks pulling most of the weight?
- Volatility: Are there sharp dips or spikes in certain assets?
- Results: Which stocks end with a higher value and which end with a lower value?

In [9]:
initial_investment_amount = 1e6

returns = calculate_returns(initial_investment_amount, dataframes, allocations)

graph_returns(returns, True, True)

     Date  Initial Amounts  Ending Amounts  Change in Value
0    AAPL         100000.0       874196.72        774196.72
1    MSFT         100000.0      1130311.82       1030311.82
2   ^IXIC         100000.0       405068.29        305068.29
3    COST         100000.0       866307.83        766307.83
4     WMT         100000.0       415206.83        315206.83
5    AMZN         100000.0      1362958.67       1262958.67
6   ^GSPC         100000.0       286297.75        186297.75
7    SBUX         100000.0       259510.73        159510.73
8     TSN         100000.0       170943.32         70943.32
9    INTC         100000.0        77142.40        -22857.60
10  Total        1000000.0      5847944.36       4847944.36


alt.LayerChart(...)

alt.LayerChart(...)

### 🧾 Portfolio Strength Breakdown
**How do you know if your portfolio is actually working?**

Think of your portfolio like a car — you wouldn’t drive it for miles without checking the engine, right? These metrics are your dashboard. They tell you whether you're cruising efficiently, burning too much fuel (aka risk), or heading toward a cliff. By regularly checking them, you’re not just investing — you’re investing intelligently. It's how you turn guessing into strategy.

These [metrics from quantstats ](https://cubed.run/blog/quantstats-elevating-portfolio-analysis-for-quants) help you assess whether your portfolio is efficient, risky, or well-balanced — and give you data to improve it over time. We can write a function to return these values for us.

In [11]:
def portfolio_stats(returns):
    # Ensure datetime is clean
    returns.set_index('Date', inplace=True)

    # Daily returns from portfolio value
    daily_returns = returns['Total'].pct_change().dropna()

    # Grab specific stats
    cagr = stats.cagr(daily_returns)
    sharpe = stats.sharpe(daily_returns)
    drawdown = stats.max_drawdown(daily_returns)
    volatility = stats.volatility(daily_returns)
    avg_return = stats.avg_return(daily_returns)

    return {"cagr":cagr, "sharpe":sharpe, "drawdown":drawdown, "volatility":volatility, "avg_return":avg_return}


Now that we've written a function for this, we can analyze the returns of our portfolio from above.

In [12]:
portfolio_info = portfolio_stats(returns)

#### Now let's analyze these metrics!

**📈 CAGR (Compound Annual Growth Rate)**
This shows the average annual growth of your portfolio over time. The higher, the better — it reflects long-term performance. A higher CAGR generally reflects strong long-term performance. For context, a CAGR of 10–15% is typically considered solid, especially when compared to market benchmarks like the S&P 500. [(Investopedia: CAGR)](https://www.investopedia.com/terms/c/cagr.asp#toc-what-is-a-good-cagr)

In [13]:
print(f"📈 CAGR (Annual Return): {portfolio_info['cagr']:.2%}")

📈 CAGR (Annual Return): 12.49%


**📊 Sharpe Ratio:**  
Measures your return per unit of risk. A Sharpe ratio above 1.0 is generally considered good. It means you're getting rewarded well for the volatility you’re taking on. A value above 1.0 suggests you're earning a good return for the amount of risk taken, while a ratio above 2.0 is considered excellent. Anything below 1.0 may indicate that returns aren’t efficiently compensating for risk. Sharpe ratios above 1 are generally considered “good," offering excess returns relative to volatility. However, investors often compare the Sharpe ratio of a portfolio or fund with those of its peers or market sector. [(Investopedia: Sharpe Ratio)](https://www.investopedia.com/terms/s/sharperatio.asp#toc-what-the-sharpe-ratio-can-tell-you)

In [14]:
print(f"📊 Sharpe Ratio: {portfolio_info['sharpe']:.2f}")

📊 Sharpe Ratio: 0.93


**📉 Max Drawdown:**  
The worst loss your portfolio experienced from peak to bottom. Lower drawdowns are better, as they reflect greater stability and less severe losses. A drawdown under 20% is generally seen as moderate, while anything over 30% may signal high risk or poor diversification. [(Investopedia: Max Drawdown)](https://www.investopedia.com/terms/d/drawdown.asp#toc-example-of-a-drawdown)

In [15]:
print(f"📉 Max Drawdown: {portfolio_info['drawdown']:.2%}")

📉 Max Drawdown: -32.34%


**📈 Volatility:**  
Reflects how much your portfolio's value fluctuates. High volatility can mean high risk — or high opportunity. Higher volatility can mean bigger gains — but also bigger losses. For comparison, broad market indices tend to have volatility in the 15–20% range; values much higher than that could mean the portfolio is more aggressive or unstable. [(Investopedia: Volatility)](https://www.investopedia.com/terms/v/volatility.asp)

In [16]:
print(f"📈 Volatility: {portfolio_info['volatility']:.2%}")

📈 Volatility: 20.77%


**📅 Average Daily Return:**  
The average return your portfolio gained (or lost) per trading day. Helps you see how it behaves short term. This captures how much the portfolio gains or loses on an average trading day. While daily gains may appear small in percentage terms, even modest positive averages can lead to significant annual growth through compounding. [(Investopedia: Average Return)](https://www.investopedia.com/terms/a/averagereturn.asp)

In [17]:
print(f"📅 Average Daily Return: {portfolio_info['avg_return']:.4%}")

📅 Average Daily Return: 0.0764%


### Monte Carlo Simulation

Now we'll run a Monte Carlo simulation to try to estimate the predicted value of the stocks in future.

Monte Carlo simulations work by determining metrics of how "random" the motion of a given thing is, which for us will be stocks. It works by simulating lots, usually thousands, of random trials on randomly generated numbers modified to the distribution of a given stock. These simulations work by finding a daily return, done by determining the change in a stock from day to day and taking a natural logarithm. Then, we calculate the "drift," or upward/downward tendency of a stock, using the equation $Drift = Average(Daily Return) - 0.5 * Variance(Daily Return)$, and a random input by using the equation $Random = StdDev(Daily Return) * random(0, 1)$, which multiplies standard deviation by a random value from 0 to 1. Finally, we sum together these two values we've calculated, and for any given day $S_i$ the price of a stock on that day of simulation is $S_i = S_{i-1} * e ^ {Drift + Random}$. [(Investopedia: Monte Carlo Simulation)](https://www.investopedia.com/terms/m/montecarlosimulation.asp) 

We will need to build up every piece of the simulation before we can put it all together and create a Monte Carlo simulation. Let's start by getting the data we'll need in a format that helps for this task.

In [18]:
def import_stock_data(tickers, start = "2015-01-01"):
    data = pd.DataFrame()
    if len(tickers) == 1:
        stock = yf.Ticker(tickers[0], session = session)
        data[tickers[0]] = stock.history(interval = '1d', start = start)['Close']
    else:
        for x in tickers:
            stock = yf.Ticker(x, session=session)
            data[x] = stock.history(interval = '1d', start = start)['Close']
    return data

Next, for the data, we aren't incredibly concerned with the actual values of a stock over time when generating our $r$ factor. Our biggest concern is how the stock changes over time. For this, we will generate two models for change over time: one that is calculated as the logarithm of the percentage change for a given stock, and one that is simply the percentage change. The simple percentage change is an easier to understand model, but the logarithmic analysis of percentage change allows us to smooth the curve of values we get, which in turn helps us minimize divergence in our models.

In [19]:
def log_returns(data):
    return np.log(1 + data.pct_change().dropna())

def simple_returns(data):
    return (data/(data.shift(1))-1)

Now that we have normalized returns for our dataset, we can use this normalized set of returns to calculate drift and volatility of a given stock. We can start with drift. Drift in a Monte Carlo simulation is the direction previous models have tended to go in, and it is given by $D = \mu - \frac{1}{2} \sigma^2$, where $\mu$ is the average value of your dataset, in this case our average daily return, and $\sigma$ is the variance of our return. This value gives the intensity and direction the overall trend of a dataset should take.

In [20]:
def drift_calc(data, return_type = "log"):
    if return_type == "log":
        lr = log_returns(data)
    elif return_type == "simple":
        lr = simple_returns(data)
    u = lr.mean()
    var = lr.var()
    drift = u-(.5*var)
    try:
        return drift.values
    except:
        return drift

Now, we are going to combine two steps, as they go very closely together. First, we are going to determine the volatility of our stock, which is given by $V = \sigma [Rand(0:1)]$, where $\sigma$ is the standard deviation of our dataset, and it is multipled with a random real number from 0 to 1. We will add this value to our drift, and that will be our $r$ value for a given day. To do this in python, we will create a matrix with a cell for every day and every iteration that initially has just a random number. We will multiply each cell by our standard deviation and then add our drift. After this, we will run an exponential calculation that determines $e^r$ if a cell's value is $r$. This matrix, which we will then return, is the $e^r$ multiplier for any given day in any iteration of a Monte Carlo simulation.

In [21]:
def daily_returns(data, days, iterations):
    ft = drift_calc(data)
    try:
        stv = log_returns(data).std().values
    except:
        stv = log_returns(data).std()

    dr = np.exp(ft + stv * norm.ppf(np.random.rand(days, iterations)))
    return dr

Now, we will finally bring back the actual values from the dataset. For every initial day in the Monte Carlo simulation, we will use the closing value of the stock last day we pulled as our first day of Monte Carlo. From here, we will be able to iterate through the matrix, and in every iteration, we will be able to multiply the previous day with the given value from our daily returns matrix that we already generated. This will give us multiple Monte Carlo iterations that we can analyze to see how we can expect a stock to behave. Some important things for our simulation include expected value, calculated as the average of all iterations of our simulation on their last day, and return percentage, which is the percentage our investment would grow by had we invested in our stock on the first day of our Monte Carlo simulation.

In [22]:
def simulate_mc(data, days, iterations):
    # generate list of e^r multipliers, initialize the price list, run sim
    returns = daily_returns(data, days, iterations)
    price_list = np.zeros_like(returns)
    price_list[0] = data.iloc[-1]
    for t in range(1, days):
        price_list[t] = price_list[t-1] * returns[t]

    total_days = days - 1

    # expected value is average of final value of stock over all simulations
    expected_value = round(pd.DataFrame(price_list).iloc[-1].mean(), 2)
    #return value is the total change in the stock from day 0 to n divided by the expected value
    return_pct = round(100*(expected_value-price_list[0,1])/expected_value, 2)

    avg_value = []
    for i in range(0, days):
        avg_value += [price_list[i].mean()]

    return [total_days, expected_value, return_pct, avg_value]

def monte_carlo(ticker, days, iterations):
    df = import_stock_data([ticker])
    return simulate_mc(df, days, iterations)

Now that we have built our functions that run a Monte Carlo simulation, we will use them on our list of tickers. For every ticker in our portfolio, we will run the Monte Carlo simulation and add it to a DataFrame, allowing us to compile a DataFrame of potential stock futures to go along with the set of stock histories we have already pulled from yfinance. Alongside the dataframe creation, we also will generate a few key pieces of information that tell us about our simulation: total days in the simulation, the expected stock value at the end of simulation, and the return percentage from day 0 to day $n$.

In [23]:
days = 50
iters = 100

mc_projections = pd.DataFrame()
mc_data = []
for x in portfolio:
    mc = monte_carlo(x, days, iters)
    mc_projections[x] = mc[3]
    mc_data += [mc[0:2]]

mc_projections['Total'] = mc_projections.sum(axis=1)

We can then use that data to plot what expected values might look like, and print the average expected value to get an idea of what future values may look like.

In [24]:
total_occurrences = mc_projections['Total'].astype(float).reset_index()

avg = mc_projections['Total'].mean()

bar_plot = alt.Chart(total_occurrences).mark_bar().encode(
    x=alt.X('Total:Q', title='Total Value').bin(maxbins=iters/8),
    y='count()'
    , tooltip=[alt.Tooltip('count()', title='Frequency')]
).properties(
    title='Frequency of Total Values in Monte Carlo Projections',
    width=600, # because the auto-size is quite small
    height=400
)

print(f"Average Expected Value is {avg:.2f}")

bar_plot.display()

Average Expected Value is 27606.77


alt.Chart(...)

### Optimizing a Stock Portfolio

It's great that we can create a stock portfolio that we can work with, and we can even analyze how strong it is using quantstats! It would be even better if we could take the portfolio we made and see if we can change our investment amounts to maximize our returns. There are a couple steps to get it set up, but thankfully, scipy has a function that can maximize our returns given an initial investment percentage for each stock, and a history of each stock's percentage change in price. We utilized [this medium article](https://medium.com/@ethan.duong1120/python-powered-portfolio-optimization-achieving-target-returns-through-weight-optimization-fc5163e5c9c6) as our framework.

First, we are going to create a sample version of our portfolio that pulls stock history for every ticker that we have selected, adding their opening price at every interval to the function and storing all of this in a dataframe. Then, we use the pct_change function to determine the change between each frame for each stock, allowing us to determine the most efficient investment of stocks.

In [31]:
## SOLUTION: Isabel's attempted code to fix the broken data insertion line below
## the line below breaking in your portfolio optimization prep section:

testList = portfolio

ticker = yf.Ticker(testList[0], session=session)
df = ticker.history(interval = '1d', start = '2015-01-01', end = '2025-04-01')
df.drop(columns=['High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], inplace=True)
df.rename({'Open' : testList[0]}, inplace=True)

for x in testList[1:]:
    ticker = yf.Ticker(x, session=session)
    data = ticker.history(interval='1d', start='2015-01-01', end='2025-04-01')
    if not data.empty:
        df = df.join(data[['Open']].rename(columns={'Open': x}), how='outer')

#calculate percent returns for each day of each stock
returns_df = df.pct_change(1).dropna()



Now, we need to determine the value of each portfolio's return. This is simply the return of each stock multipled by its weight in the portfolio. Because it is this way for each stock, we can compute the return as a dot product of returns and weights, then multiply by 250 trading days in a year to annualize our result!

In [32]:
#operationalize determining portfolio returns
def getPfReturn(weights):
    """
    return is annualized expected return of portfolio
    """
    expRetPortfolio = np.dot(np.transpose(weights), returns_df.mean()) * 250
    return expRetPortfolio

Now, we need to start bounding the function that will maximize our returns given an investment. We will use scipy's minimize function to do this. Despite its name, the minimize function works by minimizing the constraints that you give it, so minimizing the difference between our portfolio return and our target return will enable us to hit our target return, "maximizing" the output of our portfolio. If we want to truly maximize, we can continue pushing up this target return incrementally until we reach it!.

First, we will start with initial weights of each stock in our portfolio. We can either use the weights we selected previously, or we can simply start with each stock having an equal weight in our portfolio. You can choose this by slightly modifying the code below.

Now, we need to set our target return value, which we selected to arbitrarily be .2, or a 20% return. This can be modified however you wish, but do note that the minimize function will fail if maximizing to the given return percentage is impossible.

Another important goal is to bound the weight of each stock in our portfolio between 0 and 1, or 0% and 100%. It doesn't make much sense to be able to buy more than a full portfolio of stocks! We do this by creating a tuple that has the same length as our number of stocks, and each element in the tuple is another tuple storing the values (0,1), bounding our stock weights in that range

Finally, we want to create two constraints for the function. This is done using the syntax below. Our first constraint says that the sum of all weights should not exceed 1, or that our portfolio weights can't exceed 100%. We've done this already for individual stocks, but it's also important that we cant just spend 100% of our money in each stock. Finally, we also need to make sure that we "minimize" the difference between the returns for any set of portfolio weights the function generates and the target return we have set.

In [33]:
custom_weights = True  # you change to False to use equal weights

numStocks = len(returns_df.columns)

if custom_weights and allocations:
    try:
        weight_list = [float(allocations[stock]) for stock in returns_df.columns]
        total_weight = sum(weight_list)
        initialWeight = [w / total_weight for w in weight_list]
    except Exception as e:
        print("Error with custom weights. Falling back to equal weights.")
        initialWeight = [1 / numStocks] * numStocks
else:
    initialWeight = [1 / numStocks] * numStocks

    ## What I am trying to do is 1. Lets user control whether to use the user’s weights or equal weights.
    ## 2. Normalizes the weights so they sum to 1, just in case they don't sum perfectly.
    ## 3. Catch errors(like if the user forgets to fill in all weights).

# Feel free to pick a different number!
targetReturn = .2

# bounds the percentage of each stock we can hold (between 0 and 100%)
bounds = tuple((0,1) for i in range(numStocks))

# ensures the sum of all stock weights is 100% (or 1) in first constraint
# sets goal of minimize function to hit targetReturn
constraints = ({'type' : 'eq', 'fun' : lambda w : np.sum(w) - 1},
               {'type' : 'eq', 'fun' : lambda x : x.dot(returns_df.mean()) * 250 - targetReturn})

Error with custom weights. Falling back to equal weights.


Now that we have set all of our constraints, we need to use the maximize function! We set a variable called "results" equal to the minimize function with our function and all of the constraints. Through the magic of scipy, it will iterate on our weights, modifying them in an attempt to maximize the return of our portfolio. We can print results to see whether it was successful, as well as the weights of each stock in our portfolio. We can also see our returns by using our returns function on the 'x' field of our minimize output.

In [34]:
# can we reach our goal with these stocks??
results = minimize(fun=getPfReturn, x0=initialWeight, bounds=bounds, constraints=constraints)

#output
# print(results)

optimizedResults = pd.DataFrame(results['x'])

# print our optimized results
getPfReturn(weights=results["x"])
optimizedResults.index = portfolio

print(optimizedResults)

              0
AAPL   0.139523
MSFT   0.137852
^IXIC  0.088155
COST   0.125094
WMT    0.088664
AMZN   0.161632
^GSPC  0.069746
SBUX   0.084374
TSN    0.068602
INTC   0.036358


Now, we've successfully used scipy to maximize the results from our function! We can even extract the weights and use them to model a portfolio. Let's use some similar code to graph the returns of a portfolio based on an inital investment above, but using our new optimized weights!

### Optimized Portfolio Visualization

This chart shows how your portfolio performs after optimization — where the weights were adjusted to target a better return/risk profile. Each line represents an individual stock’s contribution to portfolio value over time.

Compare this plot with the initial version to see if:
  - Fewer stocks dominate the performance
  - Fluctuations are smaller (less risk)
  - The total trend is smoother or steeper

Logic Check:
A good optimization often leads to fewer volatile movements and better risk-adjusted growth.

In [35]:
initial_portfolio_val = 1e6

optimized_allocations = optimizedResults[0].to_dict()


optimized_returns = calculate_returns(initial_portfolio_val, dataframes, optimized_allocations)

graph_returns(optimized_returns, True, True)

     Date  Initial Amounts  Ending Amounts  Change in Value
0    AAPL        139523.50      1219709.85       1080186.35
1    MSFT        137852.07      1558158.28       1420306.21
2   ^IXIC         88155.02       357088.02        268933.00
3    COST        125093.60      1083695.69        958602.09
4     WMT         88663.59       368137.27        279473.68
5    AMZN        161631.87      2202975.62       2041343.75
6   ^GSPC         69745.90       199680.95        129935.04
7    SBUX         84374.04       218959.69        134585.65
8     TSN         68602.19       117270.86         48668.67
9    INTC         36358.21        28047.60         -8310.61
10  Total       1000000.00      7353723.83       6353723.83


alt.LayerChart(...)

alt.LayerChart(...)

#### 📉 Rechecking Strength After Optimization

After optimizing our portfolio allocations to achieve a target return of 40%, we reevaluate the portfolio using updated historical performance metrics.

To review:

- **CAGR (Annual Return):** Shows the expected annual growth based on optimized allocations.
- **Sharpe Ratio:** Tells us how well we’re balancing risk and reward with the new weights.
- **Max Drawdown:** Indicates if the worst-case dip has improved.
- **Volatility:** Helps assess whether the portfolio became more stable or more erratic.
- **Avg Daily Return:** Confirms whether daily trends look stronger after optimization.

This second performance check shows whether our changes actually led to a more efficient investment strategy.

In [36]:
optimized_portfolio_info = portfolio_stats(optimized_returns)

print(f"📈 CAGR (Annual Return): {optimized_portfolio_info['cagr']:.2%}")
print(f"📊 Sharpe Ratio: {optimized_portfolio_info['sharpe']:.2f}")
print(f"📉 Max Drawdown: {optimized_portfolio_info['drawdown']:.2%}")
print(f"📈 Volatility: {optimized_portfolio_info['volatility']:.2%}")
print(f"📅 Average Daily Return: {optimized_portfolio_info['avg_return']:.4%}")

📈 CAGR (Annual Return): 14.22%
📊 Sharpe Ratio: 0.98
📉 Max Drawdown: -34.44%
📈 Volatility: 22.07%
📅 Average Daily Return: 0.0863%


### Summary

After constructing our initial stock portfolio based on user-selected allocations, we evaluated its historical performance using metrics like annual return, Sharpe ratio, and drawdown. This gave us a baseline understanding of how well our portfolio performed under those weights.

We then used optimization techniques (via scipy.minimize) to automatically adjust the stock allocations in order to achieve a target return while minimizing risk. This step simulates what a financial algorithm might do to make our portfolio more efficient.

Finally, we re-evaluated the optimized portfolio using the same performance metrics. By comparing the before-and-after results, we can see whether the optimization actually improved our portfolio — either by increasing returns, reducing risk, or improving the Sharpe ratio (risk-adjusted performance).

This comparison demonstrates the power of computational thinking in making data-driven investment decisions.